In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from pathlib import Path
import tensorflow as tf

In [ ]:
#load images
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = Path ('/content/drive/MyDrive/Vgg_Images')
dataset_img = []
dataset_label = []

for e in path.iterdir():
    cat = e.relative_to('/content/drive/MyDrive/Vgg_Images')
    for f in e.iterdir():
        img = load_img(f, target_size=(256,256))
        input_arr = img_to_array(img)
        img_arr = np.reshape(input_arr,(256,256,3))
        dataset_img.append(img_arr)
        

        if str(cat) == "covid":
          dataset_label.append(0)
        if str(cat) == "normal":
          dataset_label.append(1)
        if str(cat) == "pneumonia":
          dataset_label.append(2)
        
data  = np.array(dataset_img)
print(data.shape)
data[0].shape

(1118, 256, 256, 3)


(256, 256, 3)

In [ ]:
data = data.astype('float32') / 255

from tensorflow.keras.utils import to_categorical
data_labels = to_categorical(dataset_label)


from sklearn.model_selection import train_test_split


image_train, image_test, label_train, label_test = train_test_split(data,data_labels)

In [ ]:
print(image_train.shape)
print(label_train.shape)

#image_train=image_train.reshape(840,1,256*256,3)

(838, 256, 256, 3)
(838, 3)


In [ ]:
#Vgg16 model Training
from keras.applications.vgg16 import VGG16
vgg = VGG16( input_shape=(256,256,3) ,
            classes=3, weights=None)
vgg.compile(optimizer=tf.optimizers.Adam(), 
              loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

hist=vgg.fit(image_train,label_train,epochs=6)

Epoch 1/6
27/27 [==============================] - 33s 580ms/step - loss: 1.5016 - accuracy: 0.3496
Epoch 2/6
27/27 [==============================] - 14s 507ms/step - loss: 1.1256 - accuracy: 0.3842
Epoch 3/6
27/27 [==============================] - 14s 513ms/step - loss: 1.0929 - accuracy: 0.3854
Epoch 4/6
27/27 [==============================] - 14s 506ms/step - loss: 1.0942 - accuracy: 0.3854
Epoch 5/6
27/27 [==============================] - 14s 508ms/step - loss: 1.0927 - accuracy: 0.3854
Epoch 6/6
27/27 [==============================] - 14s 511ms/step - loss: 1.0953 - accuracy: 0.3854


In [ ]:
#Vgg16 model test predictions
vgg16_predict=vgg.predict(image_test)
vgg16_predict

array([[0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.29538077, 0.32173076, 0.38288844],
       [0.

In [ ]:
test_loss,test_acc=vgg.evaluate(image_test,label_test)
print('network',test_acc)

9/9 [==============================] - 2s 184ms/step - loss: 1.1046 - accuracy: 0.3321
network 0.3321428596973419


In [ ]:
vgg19 = keras.applications.vgg19
conv_model = vgg19.VGG19(weights='imagenet', include_top=False, input_shape=(256,256,3))
for layer in conv_model.layers: 
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

full_model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adamax(lr=0.01),
                  metrics=['acc'])

history=full_model.fit(image_train,label_train,epochs=1)


80150528/80134624 [==============================] - 2s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adamax.py:90: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)


27/27 [==============================] - 7s 218ms/step - loss: 1.5979 - acc: 0.7470


In [ ]:
test_loss,test_acc=full_model.evaluate(image_test,label_test)
print('network',test_acc)

9/9 [==============================] - 2s 219ms/step - loss: 0.0043 - acc: 1.0000
network 1.0


In [ ]:
vgg19_predict=full_model.predict(image_test)
vgg19_predict

array([[6.75988037e-26, 5.04392060e-14, 1.00000000e+00],
       [6.80118091e-17, 9.99998212e-01, 1.76063520e-06],
       [1.00000000e+00, 7.22425592e-14, 1.85195674e-20],
       [1.46044163e-17, 9.99986172e-01, 1.37740171e-05],
       [1.00000000e+00, 7.26604446e-14, 1.86100015e-20],
       [1.00000000e+00, 1.30832472e-11, 1.52161478e-17],
       [1.32259563e-16, 9.99994636e-01, 5.40972997e-06],
       [9.06098937e-17, 9.99992251e-01, 7.78411140e-06],
       [1.00000000e+00, 1.51327624e-21, 1.27747569e-27],
       [3.96710759e-17, 9.99997973e-01, 1.97963300e-06],
       [1.00000000e+00, 6.59169722e-15, 1.67247731e-21],
       [8.02300334e-24, 2.30028444e-11, 1.00000000e+00],
       [2.93056156e-25, 4.83452959e-13, 1.00000000e+00],
       [2.64650746e-17, 9.99998093e-01, 1.84802491e-06],
       [1.41112289e-25, 2.93930109e-13, 1.00000000e+00],
       [3.46016395e-26, 1.70089925e-14, 1.00000000e+00],
       [1.24581316e-17, 9.99914169e-01, 8.58394851e-05],
       [1.00000000e+00, 2.11690

In [ ]:
#Vgg19 model test predictions
vgg19_predict=full_model.predict(image_test)
vgg19_predict

array([[6.75988037e-26, 5.04392060e-14, 1.00000000e+00],
       [6.80118091e-17, 9.99998212e-01, 1.76063520e-06],
       [1.00000000e+00, 7.22425592e-14, 1.85195674e-20],
       [1.46044163e-17, 9.99986172e-01, 1.37740171e-05],
       [1.00000000e+00, 7.26604446e-14, 1.86100015e-20],
       [1.00000000e+00, 1.30832472e-11, 1.52161478e-17],
       [1.32259563e-16, 9.99994636e-01, 5.40972997e-06],
       [9.06098937e-17, 9.99992251e-01, 7.78411140e-06],
       [1.00000000e+00, 1.51327624e-21, 1.27747569e-27],
       [3.96710759e-17, 9.99997973e-01, 1.97963300e-06],
       [1.00000000e+00, 6.59169722e-15, 1.67247731e-21],
       [8.02300334e-24, 2.30028444e-11, 1.00000000e+00],
       [2.93056156e-25, 4.83452959e-13, 1.00000000e+00],
       [2.64650746e-17, 9.99998093e-01, 1.84802491e-06],
       [1.41112289e-25, 2.93930109e-13, 1.00000000e+00],
       [3.46016395e-26, 1.70089925e-14, 1.00000000e+00],
       [1.24581316e-17, 9.99914169e-01, 8.58394851e-05],
       [1.00000000e+00, 2.11690

In [ ]:
resnet50 = tf.keras.applications.resnet50

conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(256,256,3))
for layer in conv_model.layers:
    layer.trainable = False
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)
full_model1 = keras.models.Model(inputs=conv_model.input, outputs=predictions)

full_model1.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adamax(lr=0.02),
                  metrics=['acc'])

history1=full_model1.fit(image_train,label_train,epochs=2)

94781440/94765736 [==============================] - 2s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adamax.py:90: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adamax, self).__init__(name, **kwargs)


Epoch 1/2
27/27 [==============================] - 8s 143ms/step - loss: 12.7145 - acc: 0.3544
Epoch 2/2
27/27 [==============================] - 3s 126ms/step - loss: 0.6636 - acc: 0.3628


In [ ]:
test_loss,test_acc=full_model1.evaluate(image_test,label_test)
print('network',test_acc)

9/9 [==============================] - 3s 213ms/step - loss: 0.6388 - acc: 0.3321
network 0.3321428596973419


In [ ]:
label_test[1]

array([0., 1., 0.], dtype=float32)

In [ ]:
#Vgg19 model test predictions
vgg19_predict=full_model.predict(image_test)
vgg19_predict

final_pred=[]
for i in vgg19_predict:
  final_pred.append(list(i).index(max(i)))

labels1=[]
for i in label_test:
  labels1.append(list(i).index(max(i)))


# from sklearn import metrics
# import matplotlib.pyplot as plt

# fpr, tpr, _ = metrics.roc_curve(labels1,  final_pred)
# auc = metrics.roc_auc_score(label_test,  label_test)

# #create ROC curve
# plt.plot(fpr,tpr,label="AUC="+str(auc))
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.legend(loc=4)
# plt.show()

In [ ]:
# data2=[]
# a=Path ('/content/drive/MyDrive/Vgg_Images/covid/218_clarius_covid_prc_convex_clean_frame0.jpg')
# img = load_img(a, target_size=(256,256))
# input_arr = img_to_array(img)
# img_arr = np.reshape(input_arr,(256,256,3))
# data2.append(img_arr)
# data1  = np.array(data2)
# print(data1.shape)
# data1[0].shape

In [ ]:
b=[2,5,3]
res=[]

for i in b:
  if i==max(b):
    res.append(1)
  else:
    res.append(0)

res


In [ ]:
c=[2,5,3]

for i in c:
  c[c.index(max(c))]=1